In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/datasource_hackathon


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
from sklearn.model_selection import train_test_split
import warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common import space
warnings.filterwarnings("ignore")

In [4]:
all_feats_df = pd.read_pickle("engineered_data/merged_data_engineered_all_features_weekly.pkl")

In [5]:
all_feats_df.shape

(1615437, 86)

In [6]:
all_feats_df = all_feats_df.drop(
    columns=["Price", "Inventory"], axis=1, errors="ignore"
)
all_feats_df.replace([np.inf, -np.inf], 1e-6, inplace=True)

In [7]:
all_feats_df[
    [
        "Client",
        "Warehouse",
        "Product",
        "monthly_inventory_category",
        "weekly_inventory_category",
        "monthly_period_inventory_category",
        "demand_category",
        "Month",
    ]
] = all_feats_df[
    [
        "Client",
        "Warehouse",
        "Product",
        "monthly_inventory_category",
        "weekly_inventory_category",
        "monthly_period_inventory_category",
        "demand_category",
        "Month",
    ]
].astype(
    "object"
)

In [8]:
# val_dt = all_feats_df.loc[all_feats_df["Tag"] == "Train", "Date"].max() - pd.DateOffset(
#     weeks=3
# )
# val_dt = str(val_dt.date())

In [9]:
train_all_feats_df = all_feats_df.loc[all_feats_df["Tag"] == "Train"].copy()
# train_df = train_all_feats_df.loc[train_all_feats_df["Date"] < val_dt].copy()
# val_df = train_all_feats_df.loc[train_all_feats_df["Date"] >= val_dt].copy()
test_all_feats_df = all_feats_df.loc[all_feats_df["Tag"] == "Test"].copy()

In [10]:
train_all_feats_df.shape, test_all_feats_df.shape

((1419748, 84), (195689, 84))

In [11]:
train_all_feats_df = train_all_feats_df.set_index(["id", "Date"])
test_all_feats_df = test_all_feats_df.set_index(["id", "Date"])

In [12]:
keep_cols = [
    "Sales_roll_mean_6_lag_13",
    "demand_category",
    "Sales_roll_median_6_lag_13",
    "Sales_ewm_alpha_08_lag_13",
    "Client",
    "Sales_lag_16",
    "Warehouse",
    "Sales_roll_max_6_lag_13",
    "Sales_ewm_alpha_09_lag_13",
    "Sales_lag_13",
    "Sales_roll_mean_5_lag_13",
    "Sales_ewm_alpha_085_lag_13",
    "Sales_roll_mean_4_lag_13",
    "Sales_ewm_alpha_08_lag_14",
    "Product",
    "Sales_roll_median_5_lag_13",
    "weekly_inventory_category",
    "Sales_roll_min_6_lag_13",
    "Month",
    "Price_roll_max_5_lag_13",
    "Sales_roll_max_5_lag_13",
    "Sales_roll_max_4_lag_13",
    "Price_ewm_alpha_08_lag_14",
    "Price_roll_mean_3_lag_13",
    "Sales_roll_median_4_lag_13",
    "Price_roll_max_6_lag_13",
    "Sales_roll_min_5_lag_13",
    "Month_cos",
    "new_combination",
    'Sales'
]

# keep_cols = train_df.columns.tolist()

In [13]:
label = 'Sales'

In [14]:
train_all_feats_df = train_all_feats_df[keep_cols]
# y_val = val_df[label]
# val_df_nolabel = val_df.drop(columns=[label])
test_all_feats_df = test_all_feats_df[[col for col in keep_cols if col != label]]

In [15]:
train_all_feats_df.shape, test_all_feats_df.shape

((1419748, 30), (195689, 29))

In [16]:
train_all_feats_df.head()

Sales_roll_mean_6_lag_13 demand_category  \
id          Date                                                   
0_100_10705 2020-07-06                  7.000702       Irregular   
            2020-07-13                  7.000980       Irregular   
            2020-07-20                  7.000921       Irregular   
            2020-07-27                  7.002056       Irregular   
            2020-08-03                  6.999836       Irregular   

                        Sales_roll_median_6_lag_13  Sales_ewm_alpha_08_lag_13  \
id          Date                                                                
0_100_10705 2020-07-06                    7.001213                        7.0   
            2020-07-13                    6.999069                        7.0   
            2020-07-20                    6.999116                        7.0   
            2020-07-27                    7.000202                        7.0   
            2020-08-03                    6.999082                        7.0   

                       Client  Sales_lag_16 Warehouse  \
id          Date                                        
0_100_10705 2020-07-06      0      7.001045       100   
            2020-07-13      0      6.999520       100   
            2020-07-20      0      7.000849       100   
            2020-07-27      0      6.999729       100   
            2020-08-03      0      7.000737       100   

                        Sales_roll_max_6_lag_13  Sales_ewm_alpha_09_lag_13  \
id          Date                                                             
0_100_10705 2020-07-06                 6.997820                        7.0   
            2020-07-13                 6.999653                        7.0   
            2020-07-20                 6.998909                        7.0   
            2020-07-27                 6.999676                        7.0   
            2020-08-03                 7.000530                        7.0   

                        Sales_lag_13  Sales_roll_mean_5_lag_13  \
id          Date                                                 
0_100_10705 2020-07-06      6.999956                  6.999061   
            2020-07-13      7.001150                  7.001336   
            2020-07-20      6.996373                  7.000144   
            2020-07-27      7.000848                  6.997926   
            2020-08-03      7.002479                  6.999862   

                        Sales_ewm_alpha_085_lag_13  Sales_roll_mean_4_lag_13  \
id          Date                                                               
0_100_10705 2020-07-06                         7.0                  7.000282   
            2020-07-13                         7.0                  7.000889   
            2020-07-20                         7.0                  6.999724   
            2020-07-27                         7.0                  6.997879   
            2020-08-03                         7.0                  7.002133   

                        Sales_ewm_alpha_08_lag_14 Product  \
id          Date                                            
0_100_10705 2020-07-06                        7.0   10705   
            2020-07-13                        7.0   10705   
            2020-07-20                        7.0   10705   
            2020-07-27                        7.0   10705   
            2020-08-03                        7.0   10705   

                        Sales_roll_median_5_lag_13 weekly_inventory_category  \
id          Date                                                               
0_100_10705 2020-07-06                    6.999089                       Low   
            2020-07-13                    6.999107                       Low   
            2020-07-20                    7.001587                       Low   
            2020-07-27                    7.000940                       Low   
            2020-08-03                    7.000210                       Low   

                   

In [17]:
# all_feats_df_train["Sales"] = np.log(all_feats_df_train["Sales"] + 1e-6)
# all_feats_df_test["Sales"] = np.log1p(all_feats_df_test["Sales"] + 1e-6)

In [18]:
metric='smape'

In [19]:
gbm_options = (
    {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
        "num_boost_round": space.Int(lower=100, upper=2000, default=200),
        "num_leaves": space.Int(lower=10, upper=50, default=36),
        "learning_rate": space.Real(0.001, 0.1, log=True),
    }
)
xgb_options = (
    {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
        "max_depth": space.Int(lower=6, upper=30, default=10),
        "eta": space.Real(0.001, 0.1, log=True),
    }
)

In [20]:
hyperparameters = {  # hyperparameters of each model type
    "GBM": gbm_options,
    "XGB": xgb_options,
    #                     "CAT":{},
}
num_trials = (
    15  # try at most 5 different hyperparameter configurations for each type of model
)
search_strategy = (
    "auto"  # to tune hyperparameters using random search routine with a local scheduler
)

hyperparameter_tune_kwargs = (
    {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
        "num_trials": num_trials,
        "scheduler": "local",
        "searcher": search_strategy,
    }
)  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_all_feats_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    # presets="good_quality",
    refit_full=True,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240917_133528"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #73~20.04.1-Ubuntu SMP Mon May 6 09:43:44 UTC 2024
CPU Count:          16
Memory Avail:       97.38 GB / 108.07 GB (90.1%)
Disk Space Avail:   5001.87 GB / 5120.00 GB (97.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality'

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240917_133528"
Train Data Rows:    1419748
Train Data Columns: 29
Label Column:       Sales
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (13681.0, 0.0, 22.86164, 134.9873)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1000

  0%|          | 0/15 [00:00<?, ?it/s]

[1000]	valid_set's l2: 3698.87	valid_set's symmetric_mean_absolute_percentage_error: -0.702136
[1000]	valid_set's l2: 4193.41	valid_set's symmetric_mean_absolute_percentage_error: -0.715124


Fitted model: LightGBM/T1 ...
	-0.7733	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	9.0s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-0.6459	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	9.16s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-0.6425	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	15.48s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-0.6405	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	5.13s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-0.6428	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	24.47s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	-0.6986	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	26.27s	 = Training   runtime
	0.09s	 = Validation runtime
Fitted model: LightG

  0%|          | 0/15 [00:00<?, ?it/s]

	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=926.82 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.

Traceback (most recent call last):
  File "/anaconda/envs/torch_env/lib/python3.10/site-packages/autogluon/core/models/abstract/model_trial.py", line 37, in model_trial
    model = fit_and_save_model(
  File "/anaconda/envs/torch_env/lib/python3.10/site-packages/autogluon/core/models/abstract/model_trial.py", line 96, in fit_and_save_model
    model.fit(**fit_args, time_limit=time_left)
  File "/anaconda/envs/torch_env/lib/python3.10/site-packages/autogluon/core/models/abstract/abstract_model.py", line 855, in 

In [21]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20240917_130706/ds_sub_fit/sub_fit_ho")

In [22]:
predictor.leaderboard(train_all_feats_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost/T4_FULL,-0.619898,NaN,symmetric_mean_absolute_percentage_error,4.616091,NaN,20.129684,4.616091,NaN,20.129684,1,True,41
1,WeightedEnsemble_L2_FULL,-0.619898,NaN,symmetric_mean_absolute_percentage_error,4.751961,NaN,20.178502,0.135869,NaN,0.048818,2,True,46
2,XGBoost/T4,-0.620201,-0.629758,symmetric_mean_absolute_percentage_error,4.831194,0.079064,30.337206,4.831194,0.079064,30.337206,1,True,18
3,WeightedEnsemble_L2,-0.620201,-0.629758,symmetric_mean_absolute_percentage_error,4.957201,0.079951,30.386024,0.126007,0.000887,0.048818,2,True,23
4,XGBoost/T8,-0.632660,-0.634980,symmetric_mean_absolute_percentage_error,4.402308,0.094063,19.454512,4.402308,0.094063,19.454512,1,True,20
5,XGBoost/T8_FULL,-0.633081,NaN,symmetric_mean_absolute_percentage_error,4.297726,NaN,13.711808,4.297726,NaN,13.711808,1,True,43
6,LightGBM/T4,-0.641257,-0.640522,symmetric_mean_absolute_percentage_error,0.380557,0.011677,5.128417,0.380557,0.011677,5.128417,1,True,4
7,LightGBM/T8_FULL,-0.642212,NaN,symmetric_mean_absolute_percentage_error,3.330596,NaN,16.413364,3.330596,NaN,16.413364,1,True,31
8,LightGBM/T8,-0.642342,-0.641700,symmetric_mean_absolute_percentage_error,3.292996,0.066927,20.946405,3.292996,0.066927,20.946405,1,True,8
9,LightGBM/T4_FULL,-0.642457,NaN,symmetric_mean_absolute_percentage_error,0.422811,NaN,4.213082,0.422811,NaN,4.213082,1,True,27


In [23]:
y_pred = predictor.predict(test_all_feats_df)
print("Predictions:  ", list(y_pred)[:5])

Predictions:   [3.418315887451172, 3.168344736099243, 3.4642767906188965, 3.455923080444336, 3.306429147720337]


In [24]:
test_all_feats_df_op = test_all_feats_df[["Client", "Warehouse", "Product"]].copy()
test_all_feats_df_op["Sales"] = y_pred
test_all_feats_df_op = test_all_feats_df_op.reset_index()
test_all_feats_df_op['Date'] = test_all_feats_df_op['Date'].dt.strftime('%Y-%m-%d')

In [25]:
test_all_feats_df_op = test_all_feats_df_op.pivot(
    index=["Client", "Warehouse", "Product"], columns="Date", values="Sales"
).reset_index()

In [26]:
test_all_feats_df_op.index.name = ''

In [27]:
test_all_feats_df_op.shape

(15053, 16)

In [28]:
test_all_feats_df_op.to_csv("outputs/test_best_feats_df_op.csv", index=False)

###################################################### END ##############################################################